In [1]:
import pandas

import os

import plotly as plotly
import plotly.figure_factory as ff
import plotly.graph_objs as go

import util as ut

from util import ESConnection
from elasticsearch_dsl import Search

es_conn = ESConnection()

project_name = os.environ.get('PROJECT', 'All')
max_time = os.environ.get('MAX_TIME', '10')


# Range of years from 2017 to the past to be used in all metrics
# Use 0 as starting point together with lt in queries to exclude 2017
analyzed_range = range(0,int(max_time))

## First project
Get the project in which auhtors made their first contribution to Mozilla. Show a ranking of top 5 projects in number of newcomers to Mozilla per year.

In [2]:
#   WHEN FILTERING BY PROJECT RESULTS ARE HIGHER THAN IF QUERY FOR ALL PROJECTS 
# BECAUSE IN THAT CASE WE COUNT FIRST COMMIT IN PROJECT AND NOT FIRTS ABSOLUTE COMMIT

#results = []
#for i in analyzed_range:

# Buckets by author name, finding first commit for each of them
s = ut.create_search(es_conn, 'git')

# Filter commits to the Project Repos
s = ut.add_project_filter(s, project_name)


# Retrieve commits before given year
s = s.filter('range', grimoire_creation_date={'lt': 'now/y'})

# EXCLUDE EMPLOYEES
s = s.exclude('terms', author_org_name=['Mozilla Staff', 'Code Sheriff'])

# Bucketize by uuid and get first and last commit
s.aggs.bucket('authors', 'terms', field='author_uuid', size=10000000) \
    .metric('first', 'top_hits', _source=['author_date', 'author_org_name', 'author_uuid', 'project'],
            size=1, sort=[{"author_date": {"order": "asc"}}]) \
    .metric('last_commit', 'max', field='author_date')
s = s.sort("author_date")
#print(s.to_dict())
result = s.execute()

authors_df = ut.get_authors_df(result, author_bucket_field='authors')
#authors_df['active_years'] = (authors_df.last_commit-authors_df.first_commit).astype('timedelta64[Y]')

# Get year of First Commit
authors_df['first_commit'] = authors_df['first_commit'].map(lambda t: t.to_pydatetime().year)

#print(len(authors_df.loc[(authors_df['first_commit'] == 2016) & (authors_df['project'] == 'Rust')]))

# Group by year of first commit and project, counting number of authors
projects_df = authors_df.groupby(['first_commit', 'project']).agg({'author': pandas.Series.nunique})
projects_df.rename(columns={"author": "# authors"}, inplace=True)
projects_df = projects_df.reset_index().sort_values(by=['first_commit', '# authors'], ascending=[False, False])

# Get top 20 projects based on newcomers from 2011
rankings_df = pandas.DataFrame()
for year in projects_df['first_commit'].unique():
    if year > 2011:
        year_df = projects_df.loc[projects_df['first_commit'] == year].head(20)
        rankings_df = pandas.concat([rankings_df, year_df]) 

ut.print_table(rankings_df, filename='git-top-projects-newcomers-table')


In [3]:
plotly.offline.init_notebook_mode(connected=True)

data = []
for project in rankings_df['project'].unique():
    #print(exp, '\n', exp_groups_evo_df.loc[exp].tolist(), '\n', exp_groups_evo_df.loc[exp].index.values)
    data.append(
        go.Scatter(
            x = rankings_df['first_commit'].unique(),
            y = rankings_df.loc[(rankings_df['project'] == project), '# authors'],
            mode = 'lines+markers',
            name = project
        )
    )
    


plotly.offline.iplot(data, filename='line-mode')

# Authors DataFrame
Get a dataframes of Authors from a given year to the past. Each dataframe includes first commit date, last commit date, org, uuid and project.

In [4]:
### DIFFERENCE BETWEEN FIRST AND LAST CONTRIBUTION
### GET SNAPSHOTS FOR DIFFERENT YEARS

results = []
for i in analyzed_range:

    # Buckets by author name, finding first commit for each of them
    s = ut.create_search(es_conn, 'git')
    
    # Filter commits to the Project Repos
    s = ut.add_project_filter(s, project_name)
    
    # EXCLUDE EMPLOYEES
    s = s.exclude('terms', author_org_name=['Mozilla Staff', 'Code Sheriff'])
    
    # Retrieve commits before given year
    s = s.filter('range', grimoire_creation_date={'lt': 'now-' + str(i) + 'y/y'})

    # Bucketize by uuid and get first and last commit
    s.aggs.bucket('authors', 'terms', field='author_uuid', size=100000) \
        .metric('first', 'top_hits', _source=['author_date', 'author_org_name', 'author_uuid', 'project'],
                size=1, sort=[{"author_date": {"order": "asc"}}]) \
        .metric('last_commit', 'max', field='author_date')
    s = s.sort("author_date")
    #print(s.to_dict())
    results.append(s.execute())


authors_dfs = []
for result in results:
    authors_df = ut.get_authors_df(result, author_bucket_field='authors')
    authors_df['active_years'] = (authors_df.last_commit-authors_df.first_commit).astype('timedelta64[Y]')
    authors_dfs.append(authors_df)

authors_dfs

[                                         author        first_commit  \
 9469   149ad981c7f8acd65b9cb9a3f306833fc4b6ec8e 2016-12-31 12:01:23   
 10973  778f8adac79c8593c8413792e9d838e0620797c1 2016-12-31 08:34:00   
 12314  f0a27b3276baf429f51f6afed8ece01f1a09e7d0 2016-12-31 05:11:58   
 10611  5c6bef06c5d1fa2e83f13db2046ec3d9d27bd00d 2016-12-30 23:09:23   
 11379  977b2b48ece5125093c2cfdedb56be00bdba4f13 2016-12-30 22:21:03   
 10533  57a1cf0d7a35c3f77475953873212148d3c01e1b 2016-12-30 21:55:02   
 8660   8c536fb5c49a281f793cb47e933fd71403891960 2016-12-30 17:26:13   
 11507  a1e6b9aca42df46460c5fed6a73cde615b2b04a3 2016-12-30 16:45:19   
 9784   26e5b20e7ccff92ec895b5f4af3f5f0b9a5b0f0a 2016-12-30 14:34:47   
 7863   09a899922407e40e04fa6ab865da040f06c0d904 2016-12-30 12:22:11   
 7568   c91f349eb811f5870fe7b4e987bd34decfbf72cb 2016-12-30 11:34:16   
 11119  829a9b7eae96736b2408c62c4b2843721116fe41 2016-12-29 23:33:52   
 11838  c19b5436d45eb06bb0d9a16accd4a16da1ad9765 2016-12-29 14:2


# Git: Newcomers by year Non-employees only

In [5]:
# AUTHORS GROUPED BY FIRST COMMIT DATE

attraction_dfs = []
for authors_df in authors_dfs:
    attraction_df = authors_df.copy()
    attraction_df['first_commit'] = attraction_df['first_commit'].apply(lambda x: str(pandas.Period(x,'A')))
    attraction_df['first_commit'] = attraction_df['first_commit'].apply(lambda x: int(x) * -1)
    attraction_df = attraction_df.groupby(['first_commit']).agg({'author': pandas.Series.nunique})
    attraction_df = attraction_df.reset_index()  
    attraction_dfs.append(attraction_df)  

attraction_dfs

[    first_commit  author
 0          -2016    2350
 1          -2015    2427
 2          -2014    2157
 3          -2013    1357
 4          -2012     896
 5          -2011     766
 6          -2010     567
 7          -2009     424
 8          -2008     386
 9          -2007     275
 10         -2006      97
 11         -2005      66
 12         -2004      50
 13         -2003      57
 14         -2002      52
 15         -2001      91
 16         -2000     113
 17         -1999      91
 18         -1998     233,     first_commit  author
 0          -2015    2427
 1          -2014    2157
 2          -2013    1357
 3          -2012     896
 4          -2011     766
 5          -2010     567
 6          -2009     424
 7          -2008     386
 8          -2007     275
 9          -2006      97
 10         -2005      66
 11         -2004      50
 12         -2003      57
 13         -2002      52
 14         -2001      91
 15         -2000     113
 16         -1999      91
 17         

In [6]:
# Plot bar charts for each dataframe
attraction_df = attraction_dfs[0]
    
plotly.offline.init_notebook_mode(connected=True)
        
data = [go.Bar(
        x=attraction_df['author'],
        y=attraction_df['first_commit'],
        orientation = 'h'
)]

layout = go.Layout(
    barmode='group',
    title=project_name + ' ' + str(2016)
)

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='horizontal-bar')   

## Years of Experience
We consider **N commits** per year, as a minimum to add one year of experience to a given author. From this assumption, we build groups of authors by years of experience. As a result, we present a plot with number of people in each group.

To give a more complete idea of how community evolves, we plot snapshots corresponding to different years. Each of them will take all commits sent until the given year, and calculate years of experience for all authors in that slice.

We are also counting authors whose last year of experience is the one we are analyzing data from. That is, if we are looking to year 2016, we only count those authors who made at least N commits in 2016. From there we add 1 year of experience for each year they fulfill this condition.

* Y axis corresponds to years of experience as defined above.
* X axis corresponds to the umber of contributors in the given group.
* Each plot shows a snapshot of this information from the specified year to the past (1998 was chosen as the oldest date to get results from). 

In [7]:
###
## GET COMMITS BY YEAR AND AUTHOR
###

results = []
min_commits = 1

for i in analyzed_range:

    # Buckets by author name, finding first commit for each of them
    s = ut.create_search(es_conn, 'git')
        
    # Filter commits to the Project Repos
    s = ut.add_project_filter(s, project_name)
    
    # Retrieve commits before given year
    s = s.filter('range', grimoire_creation_date={'lt': 'now-' + str(i) + 'y/y'})

    # Bucketize by time, uuid and organization, then count commits per year
    s.aggs.bucket('time', 'date_histogram', field='grimoire_creation_date', interval='year') \
        .bucket('authors', 'terms', field='author_uuid', size=100000, min_doc_count=min_commits) \
        .bucket('org', 'terms', field='author_org_name', size=1) \
        .metric('commits', 'cardinality', field='hash', precision_threshold=1000)

    r = s.execute()
    # In case you need to check response, uncomment line below
    #print(r.to_dict()['aggregations']['time']['buckets'])
        
    results.append(r)
    
#results


In [8]:
###
## CREATE A DF CONTAINING, FOR EACH AUTHOR UUID, COUNT OF YEARS OF EXPERIENCE (YEARS
## WITH MORE THAN N COMMITS MADE) AND LAST YEAR ACTIVE
###
exp_df_list = []
year = 2016

for result in results:
    exp_df = ut.to_df_by_time(result, 'Author', 'Time', 'Commits', 'Org', 'authors', 'time', 'commits', 'org')
    exp_df['Time'] = exp_df['Time'].apply(lambda x: str(pandas.Period(x,'A')))
    
    ## ACTIVE CONDITION #################################
    ## Filter those having less than N commits per year
    exp_df = exp_df[exp_df['Commits'] >= 1]
    #####################################################
    
    ## Group by author, get MAX YEAR and NUMBER OF ROWS FOR THE GIVEN AUTHOR
    exp_df = exp_df.groupby(['Author', 'Org']).agg({'Time': 'max', 'Commits': 'count'})
    ## Filter those whose last active year is not the one we want
    exp_df = exp_df[exp_df['Time'] == str(year)]
    
    exp_df['exp'] = exp_df['Commits']
    exp_df['last_active'] = exp_df['Time']
    exp_df= exp_df.drop('Commits', axis=1)
    exp_df = exp_df.drop('Time', axis=1)
    
    exp_df['project'] = project_name
    
    exp_df_list.append(exp_df)
    
    year -= 1

exp_df_list

[                                                        exp last_active  \
 Author                                   Org                              
 000063c4e47e93ab3b30607680609e4d2500ce5d Mozilla Staff    4        2016   
 00014f52adaaeca931c4c077a1cb9791eb769ea1 Unknown          1        2016   
 0002c6a09a45af6481c35e29c0ed7f3bdbecb3b8 Unknown          1        2016   
 000336e205e3f5e2daa6e7a0b03e612b3b2b02d3 Mozilla Staff    3        2016   
 000679e4b588906b8025d2c28d45e9ece4c88715 Unknown          1        2016   
 000a91646239261328167971b7c6d851087c3089 Unknown          1        2016   
 0012fe0f0c0ebabfb0fbed08e160f6b23cd248f8 Unknown          2        2016   
 0014e7cebbe8ef11dab9eeb5f2111e01d9eea378 Mozilla Staff    6        2016   
 00201bda17fd9d14bc1e0f8a72afbc55af090657 Unknown          1        2016   
 002893ffe1425c220756f8ba4c78e1e3bb0be50f Mozilla Staff    5        2016   
 00302655e2f88afeedfb958a0364ab13bcf7ea52 Unknown          1        2016   
 003716394a2

In [9]:
# Plot bar charts for each dataframe
i = 0
for exp_df in exp_df_list:
    if not exp_df.empty:
        ut.print_horizontal_bar_chart(exp_df, 'exp', title=project_name + ' ' + str(2016 - i), min_range=1)
        
        employees_df = exp_df[[group2 in ['Mozilla Staff', 'Code Sheriff'] for group1, group2 in exp_df.index]]
        if len(employees_df) > 0:
            ut.print_horizontal_bar_chart(employees_df, 'exp', title=project_name + ' employees ' + str(2016 - i), 
                                       min_range=1)
        
        non_employees_df = exp_df[[group2 not in ['Mozilla Staff', 'Code Sheriff'] for group1, group2 in exp_df.index]]
        if len(non_employees_df):
            ut.print_horizontal_bar_chart(non_employees_df, 'exp', 
                                       title=project_name + ' non-employees ' + str(2016 - i), min_range=1)
    i += 1
    ## GET DATA ONLY FOR 2016 (AS IT'S THE ONLY WE USE IN THE REPORT RIGHT NOW)
    break

### Evolution of Experience

Next table and plot show how each group changes over time. This way we can visualize how new people come and remain in the community. It is worth to note that we are not following a given group of people through time (it could be done following diagonals in the table, we look at this in the next section), but looking at how a given group changes from one year to another. 

For instance, if we look at the group of 2 years of experience in 2008 we see we had 204 people. If we look at the same group in 2009 we see that our **new group** of people accumulating 2 years of experience has 105 people. So, it seems we have fewer people with 2 years of experience in 2009. If we look at 2016 we find 226 people with two years of experience, so we have more people with 2 years of experience nowadays than we had 8 years ago.

Table can be read as follows:

* Cell values corresponds to the number of contributors in the given group.
* Rows corresponds to groups based on years of experience.
* Columns corresponds to years we are analyzing. 

In [10]:
exp_groups_evo_df = pandas.DataFrame(columns=['exp'])

for exp_df in exp_df_list:
    
    if exp_df.empty:
        continue
    
    year = exp_df['last_active'].unique()[0]
    exp_groups_df = pandas.DataFrame(columns=['exp', year])
    
    experience = list(range(1, int(exp_df['exp'].max()) + 1))
    
    for exp in experience:
        count = len(exp_df.loc[exp_df['exp'] == exp])
        exp_groups_df.loc[len(exp_groups_df)] = [exp, count]

    exp_groups_evo_df = exp_groups_evo_df.merge(exp_groups_df, on='exp', how='outer')


# Fill Nan with 0's
exp_groups_evo_df = exp_groups_evo_df.fillna(0)

# Reorder columns
exp_groups_evo_df = exp_groups_evo_df.set_index('exp')
exp_groups_evo_df = exp_groups_evo_df.sort_index(axis=1)


#print('Max. Exp: ', exp_groups_evo_df['exp'].max(), 'Max. Count: ')
exp_groups_evo_df

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
exp,,,,,,,,,,
1.0,392.0,505.0,551.0,724.0,1008.0,1194.0,1652.0,2405.0,2603.0,2502.0
2.0,64.0,227.0,203.0,211.0,239.0,394.0,477.0,590.0,720.0,762.0
3.0,38.0,15.0,99.0,131.0,137.0,146.0,234.0,332.0,351.0,339.0
4.0,27.0,10.0,11.0,75.0,103.0,89.0,103.0,177.0,225.0,227.0
5.0,14.0,2.0,6.0,8.0,64.0,71.0,67.0,79.0,146.0,167.0
6.0,15.0,0.0,2.0,6.0,5.0,55.0,56.0,60.0,65.0,118.0
7.0,6.0,5.0,0.0,2.0,6.0,4.0,43.0,48.0,57.0,53.0
8.0,9.0,0.0,3.0,1.0,2.0,2.0,3.0,39.0,44.0,47.0
9.0,5.0,3.0,0.0,3.0,0.0,1.0,2.0,1.0,34.0,38.0


### Employees

In [11]:
exp_groups_evo_moz_df = pandas.DataFrame(columns=['exp'])

for exp_df in exp_df_list:
    
    if exp_df.empty:
        continue
        
    exp_df = exp_df[[group2 in ['Mozilla Staff', 'Code Sheriff'] for group1, group2 in exp_df.index]]
    
    year = exp_df['last_active'].unique()[0]
    exp_groups_df = pandas.DataFrame(columns=['exp', year])
    
    experience = list(range(1, int(exp_df['exp'].max()) + 1))
    
    for exp in experience:
        count = len(exp_df.loc[exp_df['exp'] == exp])
        exp_groups_df.loc[len(exp_groups_df)] = [exp, count]

    exp_groups_evo_moz_df = exp_groups_evo_moz_df.merge(exp_groups_df, on='exp', how='outer')


# Fill Nan with 0's
exp_groups_evo_moz_df = exp_groups_evo_moz_df.fillna(0)

# Reorder columns
exp_groups_evo_moz_df = exp_groups_evo_moz_df.set_index('exp')
exp_groups_evo_moz_df = exp_groups_evo_moz_df.sort_index(axis=1)


#print('Max. Exp: ', exp_groups_evo_moz_df['exp'].max(), 'Max. Count: ')
exp_groups_evo_moz_df

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
exp,,,,,,,,,,
1.0,117.0,119.0,128.0,158.0,256.0,306.0,305.0,259.0,171.0,150.0
2.0,9.0,87.0,88.0,88.0,98.0,175.0,222.0,228.0,152.0,135.0
3.0,5.0,9.0,66.0,68.0,74.0,77.0,151.0,200.0,186.0,108.0
4.0,2.0,5.0,6.0,57.0,58.0,55.0,66.0,129.0,159.0,142.0
5.0,0.0,2.0,5.0,4.0,54.0,45.0,48.0,54.0,110.0,123.0
6.0,3.0,0.0,2.0,5.0,4.0,47.0,41.0,43.0,46.0,92.0
7.0,0.0,3.0,0.0,2.0,5.0,4.0,39.0,36.0,46.0,40.0
8.0,2.0,0.0,2.0,1.0,1.0,2.0,3.0,35.0,34.0,37.0
9.0,1.0,2.0,0.0,2.0,0.0,1.0,2.0,1.0,30.0,28.0


### Non-employees

In [12]:
exp_groups_evo_others_df = pandas.DataFrame(columns=['exp'])

for exp_df in exp_df_list:
    
    if exp_df.empty:
        continue
        
    exp_df = exp_df[[group2 not in ['Mozilla Staff', 'Code Sheriff'] for group1, group2 in exp_df.index]]
    
    if (len(exp_df) == 0):
        continue
    
    year = exp_df['last_active'].unique()[0]
    exp_groups_df = pandas.DataFrame(columns=['exp', year])
    
    experience = list(range(1, int(exp_df['exp'].max()) + 1))
    
    for exp in experience:
        count = len(exp_df.loc[exp_df['exp'] == exp])
        exp_groups_df.loc[len(exp_groups_df)] = [exp, count]

    exp_groups_evo_others_df = exp_groups_evo_others_df.merge(exp_groups_df, on='exp', how='outer')


# Fill Nan with 0's
exp_groups_evo_others_df = exp_groups_evo_others_df.fillna(0)

# Reorder columns
exp_groups_evo_others_df = exp_groups_evo_others_df.set_index('exp')
exp_groups_evo_others_df = exp_groups_evo_others_df.sort_index(axis=1)


#print('Max. Exp: ', exp_groups_evo_others_df['exp'].max(), 'Max. Count: ')
exp_groups_evo_others_df

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
exp,,,,,,,,,,
1.0,275.0,386.0,423.0,566.0,752.0,888.0,1347.0,2146.0,2432.0,2352.0
2.0,55.0,140.0,115.0,123.0,141.0,219.0,255.0,362.0,568.0,627.0
3.0,33.0,6.0,33.0,63.0,63.0,69.0,83.0,132.0,165.0,231.0
4.0,25.0,5.0,5.0,18.0,45.0,34.0,37.0,48.0,66.0,85.0
5.0,14.0,0.0,1.0,4.0,10.0,26.0,19.0,25.0,36.0,44.0
6.0,12.0,0.0,0.0,1.0,1.0,8.0,15.0,17.0,19.0,26.0
7.0,6.0,2.0,0.0,0.0,1.0,0.0,4.0,12.0,11.0,13.0
8.0,7.0,0.0,1.0,0.0,1.0,0.0,0.0,4.0,10.0,10.0
9.0,4.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,10.0


Next plot can be read as follows:
* Y axis corresponds to the number of contributors in the given group.
* X axis corresponds to years we are looking through.
* Each line corresponds to a given group based on their years of experience. 

In [13]:
plotly.offline.init_notebook_mode(connected=True)

data = []
for exp in exp_groups_evo_df.index.values:
    #print(exp, '\n', exp_groups_evo_df.loc[exp].tolist(), '\n', exp_groups_evo_df.loc[exp].index.values)
    data.append(
        go.Scatter(
            x = exp_groups_evo_df.loc[exp].index.values,
            y = exp_groups_evo_df.loc[exp].tolist(),
            mode = 'lines+markers',
            name = str(int(exp)) + ' years'
        )
    )
    


plotly.offline.iplot(data, filename='line-mode')    